Importar as bibliotecas que serão usadas:

In [10]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifie

Importar os dados treino e teste já criados:

In [11]:
train = pd.read_csv('/content/drive/My Drive/data/Titanic/train.csv')
test = pd.read_csv('/content/drive/My Drive/data/Titanic/test.csv')

Criar modelo de Machine Learning. Nesse caso, usamos o RandomForestClassifier:

In [13]:
modelo = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)

Juntar as variáveis que serão usadas no modelo:

In [14]:
variaveis = ['Sex_bin', 'Age']

Criar função para transformar os dados do 'sexo' de cada passageiro para numérico, já que o modelo de ML usado não trabalha com strings:

In [15]:
def transformar_sexo(valor) :
  if valor == 'female' :
    return 1
  else:
    return 0

train['Sex_bin'] = train['Sex'].map(transformar_sexo)

In [16]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_bin
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


Separar os dados de variáveis(x) que serão usadas no teste e os resultados esperados(y) para treinar o modelo:

In [17]:
X = train[variaveis]
Y = train['Survived']

Foi necessário tirar os dados sem informação (NaN) e para isso, de forma simples, só transformamos eles em um número sem influência, como -1:

In [18]:
X = X.fillna(-1)

Criamos o modelo:

In [19]:
modelo.fit(X,Y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

Tivemos que adaptar as informações dos dados do teste, assim como fizemos com os dados do treino:

In [20]:
test['Sex_bin'] = test['Sex'].map(transformar_sexo)

In [21]:
X_prev = test[variaveis]
X_prev = X_prev.fillna(-1)
X_prev.head()

,Sex_bin,Age
0,0,34.5
1,1,47.0
2,0,62.0
3,0,27.0
4,1,22.0


Previsão do modelo: (O problema é que ele retorna uma array, então tivemos que transformar em Series do pandas com uma cabeçalho e depois transformar em arquivo csv para enviar ao avaliador automático do Kanggle para saber como nos saimos:

In [22]:
prev = modelo.predict(X_prev)
prev

array([0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [23]:
sub = pd.Series(prev, index=test['PassengerId'], name='Survived')

In [24]:
sub.shape

(418,)

In [25]:
sub.to_csv('primeiro_modelo.csv')

In [26]:
!head -n10 primeiro_modelo.csv

PassengerId,Survived
892,0
893,1
894,0
895,1
896,1
897,0
898,1
899,0
900,1


O modelo alcançou um score de 0.71.. Isso não é muito ruim, mas não muito bom. Usando o modelo que só divide passageiros masculinos como mortos e femininos como vivos, funcionou melhor do que esse :/